In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.tokens import DocBin
from sklearn.metrics.pairwise import cosine_similarity
from numpy import mean


In [2]:
# Zeichenanzahl pro Spalte erhöhen
pd.set_option("display.max_colwidth", None)


In [3]:
# spaCy-Modell laden
nlp = spacy.load("de_core_news_lg")

# DataFrame mit Haupt-Cluster laden
df = pd.read_excel("../../data/df_de_final_duplikate_markiert_HDBSCAN.xlsx")

# spaCy-Docs laden (die du vorher vektorisiert hast)
doc_bin = DocBin().from_disk("docs_de.spacy")
docs = list(doc_bin.get_docs(nlp.vocab))


In [4]:
# Nur Fragen im HDBSCAN-Noise-Cluster
df_noise = df[df["HDBSCAN_Cluster"] == -1].copy()

# Die zugehörigen Docs holen
docs_noise = [docs[i] for i in df[df["HDBSCAN_Cluster"] == -1].index]


In [5]:
print(len(df_noise))

1538


In [6]:
noun_vectors = []

for doc in docs_noise:
    # Alle Nomen mit Vektor
    nouns = [token.vector for token in doc if token.pos_ == "NOUN" and token.has_vector]
    
    if nouns:
        noun_vector = mean(nouns, axis=0)
    else:
        noun_vector = doc.vector  # Fallback: ganzer Satzvektor
    
    noun_vectors.append(noun_vector)

noun_vectors = np.array(noun_vectors)


In [7]:
# Cosine-Similarity
similarity_matrix_nouns = cosine_similarity(noun_vectors)

# Optional: Distance-Matrix (für HDBSCAN oder DBSCAN)
distance_matrix_nouns = (1 - cosine_similarity(noun_vectors)).astype(np.float64)


In [8]:
import hdbscan


In [9]:
# HDBSCAN auf Distance-Matrix (1 - cosine similarity)
subclusterer = hdbscan.HDBSCAN(
    min_cluster_size=2,
    min_samples=1,
    metric="precomputed"
)

labels_sub = subclusterer.fit_predict(distance_matrix_nouns)


In [10]:
df_noise["Subcluster_Nomen"] = labels_sub


In [11]:
# Anzahl erkannter Subcluster anzeigen
print(df_noise["Subcluster_Nomen"].value_counts().sort_index())

# Beispiel-Fragen aus Cluster 0
df_noise[df_noise["Subcluster_Nomen"] == 154][["Frage_Text"]].head()


Subcluster_Nomen
-1      514
 0        2
 1        2
 2        4
 3        3
       ... 
 391      2
 392      4
 393      2
 394      2
 395      5
Name: count, Length: 397, dtype: int64


,Frage_Text
6,"Wie beurteilen Sie diese Aussage: ""Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen."""
3700,"Wie beurteilen Sie folgende Aussage: ""Vermögende sollen sich stärker als heute an der Finanzierung des Staates beteiligen."""


In [12]:
df_noise.loc[6]

ID_Wahl                                                                                                                          2.0
Datum                                                                                                            2019-10-20 00:00:00
Frage_ID                                                                                                                        31.0
Frage_Text            Wie beurteilen Sie diese Aussage: "Vermögende sollen sich stärker an der Finanzierung des Staates beteiligen."
Frage_Typ                                                                                                                   options7
Bereich_ID                                                                                                                    4734.0
Bereich                                                                                                                Werthaltungen
ID_gesamt                                                            

In [15]:
df_noise[(df_noise["Subcluster_Nomen"] == -1) & (df_noise["Duplikat"] == True)][["Frage_Text"]]


,Frage_Text
